In [ ]:
# Parameters
target_account = "913149361159"

In [ ]:
import boto3, os
from jupyterirtools import sso

sso.login("Jupyter-IR-AdministratorAccess", target_account)

# Cleanup
This cleanup routing will delete all resources created for any of the leaked credentials simulations. If another simulation is ongoing, it will clean out the resources for that also.

In [ ]:
resourcegroupstaggingapi_client = boto3.client('resourcegroupstaggingapi')
ec2_client2 = boto3.client('ec2')
s3_client2 = boto3.client('s3')
iam_client = boto3.client('iam')
resources = resourcegroupstaggingapi_client.get_resources(TagFilters=[
        {
            'Key': 'ir-simulation',
            'Values': [
                'leaked-credentials'
            ]
        }
    ])
for resource in resources['ResourceTagMappingList']:
    arn = resource['ResourceARN']
    
    #print(f'arn: {arn}')
    if "arn:aws:s3" in arn:
        del_bucket_name = f'{arn[arn.rindex(":")+1:]}'
        print(f"Deleting: {del_bucket_name}")
        !aws s3 rm --recursive s3://$del_bucket_name
        s3_client2.delete_bucket(Bucket=del_bucket_name)
    elif ":instance/i-" in arn:
        instance_id = arn[arn.index(":instance")+10:]
        
        try:
            terminate_response = ec2_client2.terminate_instances(InstanceIds=[instance_id])
            print(f"Terminated Instance: {instance_id}")
        except ClientError as e:
            if not e.response['Error']['Code'] == "InvalidInstanceID.NotFound":
                print(f"Error deleting instance: {e.response}")
        

In [ ]:
response = iam_client.list_roles(PathPrefix='/ir-simulation-leaked-credentials/')

for role in response['Roles']:
    role_policies = iam_client.list_attached_role_policies(RoleName=role['RoleName'])
    for policy in role_policies['AttachedPolicies']:
        iam_client.detach_role_policy(RoleName=role['RoleName'],PolicyArn=policy['PolicyArn'])
        
    iam_client.delete_role(RoleName=role['RoleName'])
    print(f"Role: {role['RoleName']} deleted")
